In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [2]:
df_merged = pd.read_csv('3 VADERphase.csv')
df_merged


,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Username,Text,Date Created,Number of Likes,Number of Retweet,Number of Replies,Source of Tweet,User ID,Conversation ID,status,Category,Translated Text,Translated Cleaned Text,Sentiment Scores,Sentiment,sentiment_label
0,2,2,2,7860,sarahsslyn,@XavierNaxa @XavierNaxa min kenapa iphone 11 s...,2021-12-30 07:53:41+00:00,6.0,0.0,2.0,Twitter for iPhone,1.476461e+18,1.476403e+18,reply,Technology,@XavierNaxa @XavierNaxa why does my iphone 11 ...,iphone ring talk people person heard talking r...,"{'neg': 0.0, 'neu': 0.859, 'pos': 0.141, 'comp...",0.2023,Positive
1,29,29,29,7889,ewanac,@XavierNaxa ip7+ boleh support smpi ios berapa...,2021-12-31 03:01:55+00:00,0.0,1.0,0.0,Twitter Web App,1.476750e+18,1.476403e+18,reply,Technology,@XavierNaxa ip7+ can support up to how many ios?,support ios,"{'neg': 0.0, 'neu': 0.27, 'pos': 0.73, 'compou...",0.4019,Positive
2,32,32,32,7892,maisaradina1,@XavierNaxa Wswwqaaqqaaqaaqaaqqaqaaaqqaqsq,2021-12-31 03:02:30+00:00,0.0,0.0,0.0,Twitter for Android,1.476751e+18,1.476403e+18,reply,Technology,@XavierNaxa Wow,wow,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.5859,Positive
3,33,33,33,7893,serilee_,@XavierNaxa @hanisofea19 @bbabygallll haha,2021-12-31 04:55:34+00:00,0.0,0.0,0.0,Twitter for iPhone,1.476779e+18,1.476403e+18,reply,Technology,@XavierNaxa @hanisofea19 @bbabygallll haha,haha,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.4588,Positive
4,34,34,34,7894,ahmadalhedra,@XavierNaxa Tahniah anda buat pilihan yg tepat...,2021-12-31 05:18:35+00:00,0.0,0.0,1.0,Twitter for iPhone,1.476785e+18,1.476403e+18,reply,Technology,@XavierNaxa Congratulations you made the right...,congratulations choice,"{'neg': 0.0, 'neu': 0.204, 'pos': 0.796, 'comp...",0.5994,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57210,115104,115104,115104,9662,SyedSaddiq,Sebab itu saya pertahankan mandat rakyat sampa...,2021-10-30 15:27:18+00:00,187.0,39.0,75.0,Twitter for iPhone,1.454470e+18,1.454470e+18,Main post,Politics,That's why I defend the people's mandate until...,defend people mandate today power broker throw...,"{'neg': 0.088, 'neu': 0.615, 'pos': 0.296, 'co...",0.6597,Positive
57211,115106,115106,115106,10900,SyedSaddiq,I’m so confused.\n\nBoleh ke tak ni buat progr...,2021-10-24 05:03:35+00:00,3347.0,2608.0,150.0,Twitter for iPhone,1.452139e+18,1.452139e+18,Main post,Politics,I'm so confused.\n\nIs it possible to hold a l...,confused hold large scale program closed hall ...,"{'neg': 0.187, 'neu': 0.813, 'pos': 0.0, 'comp...",-0.3182,Negative
57212,115107,115107,115107,11142,SyedSaddiq,"Banyak kali saya suarakan.\n\nSebab itu,\n\n1)...",2021-10-22 10:33:57+00:00,1495.0,482.0,70.0,Twitter for iPhone,1.451497e+18,1.451497e+18,Main post,Politics,"Many times I voiced it.\n\nBecause of that,\n\...",times voiced minister salary cut reject discou...,"{'neg': 0.251, 'neu': 0.645, 'pos': 0.104, 'co...",-0.6249,Negative
57213,115108,115108,115108,11445,SyedSaddiq,“Tertuduh yang juga penjual ikan didakwa memil...,2021-10-17 15:00:44+00:00,1395.0,591.0,54.0,Twitter for iPhone,1.449752e+18,1.449752e+18,Main post,Politics,"""The accused, who is also a fish seller, is ac...",accused fish seller accused possessing drug sy...,"{'neg': 0.409, 'neu': 0.395, 'pos': 0.196, 'co...",-0.7941,Negative


In [3]:
df_entrepreneur = df_merged[df_merged["Category"] == "Entrepreneur"]
df_entertainer = df_merged[df_merged["Category"] == "Entertainer"]
df_technology = df_merged[df_merged["Category"] == "Technology"]
df_celebrity = df_merged[df_merged["Category"] == "Celebrity"]
df_politics = df_merged[df_merged["Category"] == "Politics"]
df_others = df_merged[df_merged["Category"] == "Fitness and others"]


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Replace NaN values with an empty string
df_merged['Translated Cleaned Text'] = df_merged['Translated Cleaned Text'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10,1000],
    'clf__gamma': [0.01,0.02,0.03,0.04,0.05,0.06,0.1, 1],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into training and testing sets
X_train_merge, X_test_merge, y_train_merge, y_test_merge = train_test_split(df_merged['Translated Cleaned Text'], 
                                                                df_merged['Sentiment'].apply(lambda score: 1 if score > 0 else 0),
                                                                test_size=0.2, random_state=42)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_merge, y_train_merge)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")



Best hyperparameters: {'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'linear', 'vect__ngram_range': (1, 1)}
Best score: 0.9579436183123964


In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_merge, y_train_merge)

# Use the final model to make predictions on the test set
y_pred_merge = pipeline.predict(X_test_merge)

# Evaluate the performance of the final model on the test set
accuracy_merge = accuracy_score(y_test_merge, y_pred_merge)
precision_merge = precision_score(y_test_merge, y_pred_merge)
recall_merge = recall_score(y_test_merge, y_pred_merge)
f1_merge = f1_score(y_test_merge, y_pred_merge)

print(f"Accuracy: {accuracy_merge}")
print(f"Precision: {precision_merge}")
print(f"Recall: {recall_merge}")
print(f"F1 score: {f1_merge}")


Accuracy: 0.9636458970549681
Precision: 0.9705799151343706
Recall: 0.9705799151343706
F1 score: 0.9705799151343706


In [6]:
output_merge = pd.DataFrame({'text': X_test_merge, 'vader_label': y_test_merge, 'predicted_label': y_pred_merge})
output_merge.to_csv("svm_merge.csv")

In [7]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_merged.loc[y_test_merge.index, 'SVM Predicted Label'] = y_pred_merge

# Save the updated DataFrame to a CSV file
df_merged.to_csv('updated_df_merged.csv', index=False)


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Replace NaN values with an empty string
df_technology['Translated Cleaned Text'] = df_technology['Translated Cleaned Text'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10,1000],
    'clf__gamma': [0.01,0.02,0.03,0.04,0.05,0.06,0.1, 1],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into training and testing sets
X_train_tech, X_test_tech, y_train_tech, y_test_tech = train_test_split(df_technology['Translated Cleaned Text'], 
                                                                df_technology['Sentiment'].apply(lambda score: 1 if score > 0 else 0),
                                                                test_size=0.2, random_state=42)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_tech, y_train_tech)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")


C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\2761322868.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_technology['Translated Cleaned Text'] = df_technology['Translated Cleaned Text'].fillna('')


Best hyperparameters: {'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'linear', 'vect__ngram_range': (1, 1)}
Best score: 0.8403827639525046


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_tech, y_train_tech)

# Use the final model to make predictions on the test set
y_pred_tech = pipeline.predict(X_test_tech)

# Evaluate the performance of the final model on the test set
accuracy_tech = accuracy_score(y_test_tech, y_pred_tech)
precision_tech = precision_score(y_test_tech, y_pred_tech)
recall_tech = recall_score(y_test_tech, y_pred_tech)
f1_tech = f1_score(y_test_tech, y_pred_tech)

print(f"Accuracy: {accuracy_tech}")
print(f"Precision: {precision_tech}")
print(f"Recall: {recall_tech}")
print(f"F1 score: {f1_tech}")


Accuracy: 0.8661971830985915
Precision: 0.8908629441624365
Recall: 0.9140625
F1 score: 0.9023136246786632


In [10]:
output_tech = pd.DataFrame({'text': X_test_tech, 'vader_label': y_test_tech, 'predicted_label': y_pred_tech})
output_tech.to_csv("svm_tech.csv")

In [11]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_technology.loc[y_test_tech.index, 'SVM Predicted Label'] = y_pred_tech

# Save the updated DataFrame to a CSV file
df_technology.to_csv('updated_df_tech.csv', index=False)


C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\2872777365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_technology.loc[y_test_tech.index, 'SVM Predicted Label'] = y_pred_tech


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Replace NaN values with an empty string
df_politics['Translated Cleaned Text'] =df_politics['Translated Cleaned Text'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10,1000],
    'clf__gamma': [0.01,0.1, 1, 10,100],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into training and testing sets
X_train_pol, X_test_pol, y_train_pol, y_test_pol = train_test_split(df_politics['Translated Cleaned Text'], 
                                                                df_politics['Sentiment'].apply(lambda score: 1 if score > 0 else 0),
                                                                test_size=0.2, random_state=42)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_pol, y_train_pol)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\1809369241.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_politics['Translated Cleaned Text'] =df_politics['Translated Cleaned Text'].fillna('')


Best hyperparameters: {'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'linear', 'vect__ngram_range': (1, 1)}
Best score: 0.9480161590835771


In [13]:
# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_pol, y_train_pol)

# Use the final model to make predictions on the test set
y_pred_pol = pipeline.predict(X_test_pol)

# Evaluate the performance of the final model on the test set
accuracy_pol = accuracy_score(y_test_pol, y_pred_pol)
precision_pol = precision_score(y_test_pol, y_pred_pol)
recall_pol = recall_score(y_test_pol, y_pred_pol)
f1_pol = f1_score(y_test_pol, y_pred_pol)

print(f"Accuracy: {accuracy_pol}")
print(f"Precision: {precision_pol}")
print(f"Recall: {recall_pol}")
print(f"F1 score: {f1_pol}")


Accuracy: 0.9532761957423279
Precision: 0.9576291359200191
Recall: 0.9617499402342816
F1 score: 0.9596851145038168


In [14]:
output_pol = pd.DataFrame({'text': X_test_pol, 'vader_label': y_test_pol, 'predicted_label': y_pred_pol})
output_pol.to_csv("svm_politic.csv")

In [15]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_politics.loc[y_test_pol.index, 'SVM Predicted Label'] = y_pred_pol

# Save the updated DataFrame to a CSV file
df_politics.to_csv('updated_df_pol.csv', index=False)


C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\496997231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_politics.loc[y_test_pol.index, 'SVM Predicted Label'] = y_pred_pol


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Replace NaN values with an empty string
df_entrepreneur['Translated Cleaned Text'] =df_entrepreneur['Translated Cleaned Text'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10,1000],
    'clf__gamma': [0.01,0.1, 1, 10,100],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into training and testing sets
X_train_entrep, X_test_entrep, y_train_entrep, y_test_entrep = train_test_split(df_entrepreneur['Translated Cleaned Text'], 
                                                                df_entrepreneur['Sentiment'].apply(lambda score: 1 if score > 0 else 0),
                                                                test_size=0.2, random_state=42)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_entrep, y_train_entrep)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\829794100.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entrepreneur['Translated Cleaned Text'] =df_entrepreneur['Translated Cleaned Text'].fillna('')


Best hyperparameters: {'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'linear', 'vect__ngram_range': (1, 1)}
Best score: 0.9127351221439023


In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_entrep, y_train_entrep)

# Use the final model to make predictions on the test set
y_pred_entrep = pipeline.predict(X_test_entrep)

# Evaluate the performance of the final model on the test set
accuracy_entrep = accuracy_score(y_test_entrep, y_pred_entrep)
precision_entrep = precision_score(y_test_entrep, y_pred_entrep)
recall_entrep = recall_score(y_test_entrep, y_pred_entrep)
f1_entrep = f1_score(y_test_entrep, y_pred_entrep)

print(f"Accuracy: {accuracy_entrep}")
print(f"Precision: {precision_entrep}")
print(f"Recall: {recall_entrep}")
print(f"F1 score: {f1_entrep}")

Accuracy: 0.9144838212634823
Precision: 0.9291084854994629
Recall: 0.9505494505494505
F1 score: 0.939706681151548


In [18]:
output_entrep = pd.DataFrame({'text': X_test_entrep, 'vader_label': y_test_entrep, 'predicted_label': y_pred_entrep})
output_entrep.to_csv("svm_entrep.csv")

In [19]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_entrepreneur.loc[y_test_entrep.index, 'SVM Predicted Label'] = y_pred_entrep

# Save the updated DataFrame to a CSV file
df_entrepreneur.to_csv('updated_df_entrep.csv', index=False)

C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\2966863844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entrepreneur.loc[y_test_entrep.index, 'SVM Predicted Label'] = y_pred_entrep


In [20]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Replace NaN values with an empty string
df_celebrity['Translated Cleaned Text'] =df_celebrity['Translated Cleaned Text'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10,1000],
    'clf__gamma': [0.01,0.1, 1, 10,100],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into training and testing sets
X_train_celeb, X_test_celeb, y_train_celeb, y_test_celeb = train_test_split(df_celebrity['Translated Cleaned Text'], 
                                                                df_celebrity['Sentiment'].apply(lambda score: 1 if score > 0 else 0),
                                                                test_size=0.2, random_state=42)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_celeb, y_train_celeb)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\4272066955.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_celebrity['Translated Cleaned Text'] =df_celebrity['Translated Cleaned Text'].fillna('')


Best hyperparameters: {'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'linear', 'vect__ngram_range': (1, 1)}
Best score: 0.8730925352597179


In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_celeb, y_train_celeb)

# Use the final model to make predictions on the test set
y_pred_celeb = pipeline.predict(X_test_celeb)

# Evaluate the performance of the final model on the test set
accuracy_celeb = accuracy_score(y_test_celeb, y_pred_celeb)
precision_celeb = precision_score(y_test_celeb, y_pred_celeb)
recall_celeb = recall_score(y_test_celeb, y_pred_celeb)
f1_celeb = f1_score(y_test_celeb, y_pred_celeb)

print(f"Accuracy: {accuracy_celeb}")
print(f"Precision: {precision_celeb}")
print(f"Recall: {recall_celeb}")
print(f"F1 score: {f1_celeb}")

Accuracy: 0.9154929577464789
Precision: 0.9186046511627907
Recall: 0.9753086419753086
F1 score: 0.9461077844311376


In [22]:
output_celeb = pd.DataFrame({'text': X_test_celeb, 'vader_label': y_test_celeb, 'predicted_label': y_pred_celeb})
output_celeb.to_csv("svm_celeb.csv")

In [23]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_celebrity.loc[y_test_celeb.index, 'SVM Predicted Label'] = y_pred_celeb

# Save the updated DataFrame to a CSV file
df_celebrity.to_csv('updated_df_celeb.csv', index=False)

C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\3929145901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_celebrity.loc[y_test_celeb.index, 'SVM Predicted Label'] = y_pred_celeb


In [24]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Replace NaN values with an empty string
df_entertainer['Translated Cleaned Text'] =df_entertainer['Translated Cleaned Text'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10,1000],
    'clf__gamma': [0.01,0.1, 1, 10,100],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into training and testing sets
X_train_enter, X_test_enter, y_train_enter, y_test_enter = train_test_split(df_entertainer['Translated Cleaned Text'], 
                                                                df_entertainer['Sentiment'].apply(lambda score: 1 if score > 0 else 0),
                                                                test_size=0.2, random_state=42)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_enter, y_train_enter)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\611527851.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entertainer['Translated Cleaned Text'] =df_entertainer['Translated Cleaned Text'].fillna('')


Best hyperparameters: {'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'linear', 'vect__ngram_range': (1, 2)}
Best score: 0.9258765979738011


In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_enter, y_train_enter)

# Use the final model to make predictions on the test set
y_pred_enter = pipeline.predict(X_test_enter)

# Evaluate the performance of the final model on the test set
accuracy_enter = accuracy_score(y_test_enter, y_pred_enter)
precision_enter = precision_score(y_test_enter, y_pred_enter)
recall_enter = recall_score(y_test_enter, y_pred_enter)
f1_enter = f1_score(y_test_enter, y_pred_enter)

print(f"Accuracy: {accuracy_enter}")
print(f"Precision: {precision_enter}")
print(f"Recall: {recall_enter}")
print(f"F1 score: {f1_enter}")

Accuracy: 0.9269430051813472
Precision: 0.927892234548336
Recall: 0.959049959049959
F1 score: 0.9432138542086187


In [26]:
output_enter = pd.DataFrame({'text': X_test_enter, 'vader_label': y_test_enter, 'predicted_label': y_pred_enter})
output_enter.to_csv("svm_enter.csv")

In [27]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_entertainer.loc[y_test_enter.index, 'SVM Predicted Label'] = y_pred_enter

# Save the updated DataFrame to a CSV file
df_entertainer.to_csv('updated_df_enter.csv', index=False)

C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\659843509.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entertainer.loc[y_test_enter.index, 'SVM Predicted Label'] = y_pred_enter


In [28]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Replace NaN values with an empty string
df_others['Translated Cleaned Text'] =df_others['Translated Cleaned Text'].fillna('')

# Define the pipeline with CountVectorizer, SVM classifier, and parameter grid for hyperparameter tuning
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC()),
])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__kernel': ('linear', 'rbf'),
    'clf__C': [0.1, 1, 10,1000],
    'clf__gamma': [0.01,0.1, 1, 10,100],
}

# Define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)

# Split the data into training and testing sets
X_train_others, X_test_others, y_train_others, y_test_others = train_test_split(df_others['Translated Cleaned Text'], 
                                                                df_others['Sentiment'].apply(lambda score: 1 if score > 0 else 0),
                                                                test_size=0.2, random_state=42)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_others, y_train_others)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\573671987.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_others['Translated Cleaned Text'] =df_others['Translated Cleaned Text'].fillna('')


Best hyperparameters: {'clf__C': 1000, 'clf__gamma': 0.01, 'clf__kernel': 'rbf', 'vect__ngram_range': (1, 1)}
Best score: 0.8185185185185185


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Fit a final model on the entire training set using the best hyperparameters
pipeline.set_params(**grid_search.best_params_)
pipeline.fit(X_train_others, y_train_others)

# Use the final model to make predictions on the test set
y_pred_others = pipeline.predict(X_test_others)

# Evaluate the performance of the final model on the test set
accuracy_others = accuracy_score(y_test_others, y_pred_others)
precision_others = precision_score(y_test_others, y_pred_others)
recall_others = recall_score(y_test_others, y_pred_others)
f1_others = f1_score(y_test_others, y_pred_others)

print(f"Accuracy: {accuracy_others}")
print(f"Precision: {precision_others}")
print(f"Recall: {recall_others}")
print(f"F1 score: {f1_others}")

Accuracy: 0.7635467980295566
Precision: 0.7737226277372263
Recall: 0.8617886178861789
F1 score: 0.8153846153846154


In [30]:
output_others = pd.DataFrame({'text': X_test_others, 'vader_label': y_test_others, 'predicted_label': y_pred_others})
output_others.to_csv("svm_others.csv")

In [31]:
# Create a new column in the test data portion of df_merged and assign the predicted labels to it
df_others.loc[y_test_others.index, 'SVM Predicted Label'] = y_pred_others

# Save the updated DataFrame to a CSV file
df_others.to_csv('updated_df_others.csv', index=False)

C:\Users\Fikri\AppData\Local\Temp\ipykernel_16080\3208066785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_others.loc[y_test_others.index, 'SVM Predicted Label'] = y_pred_others
